In [64]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio
import rasterio.mask
import glob
import os.path

In [65]:
# *** Variables ***
src_p = "/home/jestopin/PHD/data/human_footprint/WGS84/*.tif"
dest_p = '/home/jestopin/PHD/data/human_footprint/WGS84_crop_HF'
# Bounding box 
min_x, min_y, max_x, max_y = -32.26344,  26.63842,  35.58677,  72.18392
# *****************

src_paths = glob.glob(src_p)
print(len(src_paths))
src_paths

18


['/home/jestopin/PHD/data/human_footprint/WGS84/NavWater2009_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/croplands1992_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/Popdensity1990_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/HFP2009_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/Railways_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/HFP1993_int_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/croplands2005_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/Popdensity2010_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/NavWater1994_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/Built1994_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/Built2009_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/Pasture2009_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/HFP2009_int_WGS84.tif',
 '/home/jestopin/PHD/data/human_footprint/WGS84/Roads_WGS84.tif

In [66]:
# WGS84 Bounding box
lon_list = [min_x, max_x, max_x, min_x, min_x]
lat_list = [min_y, min_y, max_y, max_y, min_y]

polygon_geom = Polygon(zip(lon_list, lat_list))
polygon = gpd.GeoDataFrame(crs='epsg:4326', geometry=[polygon_geom])

In [69]:
for path in src_paths:
    
    var_name = path.split("/")[-1][:-4]
    print(var_name)
    
   # Reads src raster
    with rasterio.open(path) as src:
        # Metadata
        out_meta = src.meta
        # Adapts bounding box to src CRS
        polygon_CRS = polygon.to_crs(out_meta['crs'])
        # Only reads cropped data
        out_image, out_transform = rasterio.mask.mask(src, polygon_CRS['geometry'], crop=True)
        
    
    # Prepares out metadata
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                     "compress": "deflate"})
    
    # Writes out raster
    out_p = os.path.join(dest_p,var_name+'.tif')
    with rasterio.open(out_p, "w", **out_meta) as dest:
        dest.write(out_image)

NavWater2009_WGS84
croplands1992_WGS84
Popdensity1990_WGS84
HFP2009_WGS84
Railways_WGS84
HFP1993_int_WGS84
croplands2005_WGS84
Popdensity2010_WGS84
NavWater1994_WGS84
Built1994_WGS84
Built2009_WGS84
Pasture2009_WGS84
HFP2009_int_WGS84
Roads_WGS84
Pasture1993_WGS84
HFP1993_WGS84
Lights2009_WGS84
Lights1994_WGS84


- Verif

In [72]:
raster = rasterio.open(path)
# raster = rasterio.open('/home/jestopin/PHD/data/human_footprint/Dryadv3/Maps/Built1994.tif')
print(raster.profile)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 127.0, 'width': 36390, 'height': 15683, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.009892734568157884, 0.0, -179.99999995447251,
       0.0, -0.009892734568157884, 89.8976232152096), 'blockysize': 1, 'tiled': False, 'interleave': 'band'}
